<a href="https://colab.research.google.com/github/jpbasco/challengeTelecomX/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [1]:
import pandas as pd

In [2]:
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
datos = pd.read_json(url)
#datos.info()
#datos.head()

#🔧 Transformación

In [3]:
norm_customer = pd.json_normalize(datos['customer'])
norm_phone = pd.json_normalize(datos['phone'])
norm_internet = pd.json_normalize(datos['internet'])
norm_account = pd.json_normalize(datos['account'])

datos = datos.drop(columns=['customer']).join(norm_customer)
datos = datos.drop(columns=['phone']).join(norm_phone)
datos = datos.drop(columns=['internet']).join(norm_internet)
datos = datos.drop(columns=['account']).join(norm_account)

datos.head()


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [4]:
datos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [7]:
datos.to_csv('datos_limpios.csv', index=False)

In [ ]:
valores_churn = pd.unique(datos['Churn'])
#valores_churn
datos['Churn'] = datos['Churn'].replace('','Sin informacion')
valores_churn

In [ ]:
valores_PaymetMethod = pd.unique(datos['PaymentMethod'])
valores_PaymetMethod

In [ ]:
valores_ChargesTotal = pd.unique(datos['Charges.Total'])
valores_ChargesTotal


In [ ]:
datos['Charges.Total'] = (datos['Charges.Total'].replace(' ', 0))

In [ ]:
datos['Cuentas_Diarias'] = datos['Charges.Monthly'] / 30
datos.head()
#datos.info()

#📊 Carga y análisis

In [ ]:
datos.describe()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
conteo = datos['Churn'].value_counts()
conteo

In [ ]:
conteo.plot(kind='bar')
plt.title('Distribucion de evasion')
plt.xlabel('Es evasor?')
plt.ylabel('Cantidad de clientes')
plt.show()


In [ ]:
data_churn = datos[['Churn','gender','Contract','PaymentMethod','Charges.Total','tenure']]
data_churn
#data_churn.describe()

In [ ]:
conteo_genero = pd.crosstab(data_churn['Churn'], data_churn['gender'])
conteo_genero

In [ ]:
conteo_contrato = pd.crosstab(data_churn['Churn'], data_churn['Contract'])
conteo_contrato

In [ ]:
conteo_contrato.plot(kind='bar')

plt.title("Evasion por tipo de contrato")
plt.xlabel("Es evasor?")
plt.ylabel("Cantidad")
plt.legend(title="Contrato")
plt.show()


In [ ]:
conteo_pago = pd.crosstab(data_churn['Churn'], data_churn['PaymentMethod'])
conteo_pago

In [ ]:
conteo_pago.plot(kind='bar')

plt.title("Evasion por metodo de pago")
plt.xlabel("Es evasor?")
plt.ylabel("Cantidad")
plt.legend(title="Metodo de pago")
plt.show()

In [ ]:
conteo_tiempo = pd.crosstab(data_churn['Churn'], data_churn['tenure'])
conteo_gastado

In [ ]:
plt.figure(figsize=(12,6))

for churn_status in conteo_tiempo.index:
    plt.plot(conteo_tiempo.columns, conteo_tiempo.loc[churn_status], marker='o', label=churn_status)

plt.title('Distribución de Churn según Tenure')
plt.xlabel('Tenure (meses)')
plt.ylabel('Cantidad de clientes')
plt.legend(title='Churn')
plt.grid(True)
plt.show()

#📄Informe final

Análisis de Evasión de Clientes – Telecom X

Se realizó un análisis detallado sobre la evasión de clientes de Telecom X, entendida como la acción de dar de baja el servicio. Para ello, se siguió un proceso estructurado de ETL (Extract, Transform, Load):

Extracción (Extract): Se obtuvieron los datos de distintas fuentes, incluyendo registros de clientes, pagos, y estado de suscripción.

Transformación (Transform): Se limpiaron y normalizaron los datos, resolviendo inconsistencias, convirtiendo tipos de datos, manejando valores faltantes y aplanando estructuras complejas como diccionarios y listas dentro de columnas. Esto permitió generar un DataFrame homogéneo, listo para el análisis.

Carga (Load): Los datos transformados se cargaron en un formato único y consistente, apto para el análisis exploratorio y la generación de reportes.

Durante el análisis, se exploraron patrones de baja de servicio en función de variables como duración del contrato (tenure), método de pago y tipo de suscripción. Se identificaron tendencias claras en la evasión de clientes, permitiendo determinar factores de riesgo asociados a la pérdida de clientes.

Recomendaciones

A partir de los resultados obtenidos, se sugiere implementar las siguientes acciones para reducir la evasión:

Evitar contratos mes-a-mes: Los clientes con contratos mensuales presentan mayor probabilidad de dar de baja el servicio; incentivar planes de mayor duración puede aumentar la retención.

Evitar pagos mediante cheques electrónicos: Este método de pago se asocia con mayor tasa de evasión; promover métodos de pago más seguros y automáticos podría reducir las bajas.